In [42]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F


class Critic(nn.Module):
    def __init__(self, args, n_theta, n_classes=5):
        super(Critic, self).__init__()
        self.__args = args
        self.__n_classes = n_classes
        self.__n_theta = n_theta
        self.__kernel_size = 2
        
        for i in range(5):
            setattr(self, f'_conv{i}', nn.Conv1d(
                in_channels=i * 8 + 1,
                out_channels=8,
                kernel_size=self.__kernel_size,
                dilation=2 ** i,
                padding=math.floor(2 ** (i - 1) * (self.__kernel_size - 1))
            )) # padding?, activation function?
        
    def conv_forward(self, x):
        x = x.unsqueeze(1) # add channels
        
        for i in range(5):
            layer = getattr(self, f'_conv{i}')
            y = F.relu(layer(x))
            print(x.shape)
            print(y.shape)
            x = torch.cat((x, y), 1)
        return x
        
    def forward(self, predictions, theta):
        pass

In [43]:
x = torch.randn(2, 10 + 4)

c = Critic({}, 10)
c.conv_forward(x)

torch.Size([2, 1, 14])
torch.Size([2, 8, 13])


RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 1. Got 14 and 13 in dimension 2 at /opt/conda/conda-bld/pytorch_1570710853631/work/aten/src/TH/generic/THTensor.cpp:689